In [ ]:
pip install pandas transformers datasets torch peft bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.5/158.5 kB 15.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from huggingface_hub import login
login(token = "your_HuggingFace_token")

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model
import torch

In [ ]:
# Load the dataset
df = pd.read_csv("mahabharat_1-2.csv")
# Convert all necessary columns to strings before concatenation
df["prompt"] = df["Section"].astype(str) + " - " + df["Parv"].astype(str) + ": " + df["Key Event"].astype(str)
# Rename "Summary" to "story"
df = df.rename(columns = {"Summary": "story"})
# Drop unnecessary columns (optional)
df = df[["prompt", "story"]]
# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df)
print(dataset[0])  # Display the first example

{'prompt': '1 - Adi Parv: Maharaja Shantanu Marries the Celestial Ganga', 'story': 'According to the historical records of this earth, there once lived a King named Maharaja Shantanu, the son of Pratipa, who took his birth in the solar dynasty and was considered naradeva, the manifest representative of the Supreme Lord on earth. His fame and rule extended to all parts of the world. The qualities of self-control, liberality, forgiveness, intelligence, modesty, patience and power always resided this exalted emperor. His neck was marked with three lines like a conchshell, and his shoulders were broad. In prowess He resembled a maddened elephant. Above all these qualities, he was a devoted servant of Lord Vishnu, and therefore he was given the title, "King of kings".\r\nOnce when Maharaja Shantanu, that bull among men, was wandering in the forest, he came upon a place frequented by the Siddhas and Charanas (a class of heavenly demigods). There he saw an angelic woman who appeared like the 

In [ ]:
model_name = "mistralai/Mistral-7B-Instruct-v0.3"
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Safely load the model and initialize weights
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="balanced_low_0").to_empty(device="cuda" if torch.cuda.is_available() else "cpu")
# Apply LoRA Adaptation
config = LoraConfig(r=16, lora_alpha = 32, target_modules = ["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj"], lora_dropout = 0.05, bias = "none")
model = get_peft_model(model, config)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Use `eos_token` as padding

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
def tokenize_function(examples):
    # Concatenate "prompt" and "story" for each example in the batch
    texts = [p + " " + s for p, s in zip(examples["prompt"], examples["story"])]
    return tokenizer(texts, truncation = True, padding = "max_length", max_length = 512)

tokenized_datasets = dataset.map(tokenize_function, batched = True)

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir = "./mistral_mythology",
    evaluation_strategy = "epoch",
    learning_rate = 2e-5,
    per_device_train_batch_size = 1,  # Reduce batch size for large models
    per_device_eval_batch_size = 1,
    num_train_epochs = 3,
    weight_decay = 0.01,
    save_steps = 500,
    save_total_limit = 2,
    logging_dir = "./logs",
    push_to_hub = True
)
# Define the data collator for causal language modeling
data_collator = DataCollatorForLanguageModeling(tokenizer = tokenizer, mlm = False)  # Set to False because this is causal language modeling

In [ ]:
# Split the dataset into training and evaluation datasets
train_test_split = tokenized_datasets.train_test_split(test_size = 0.1)  # 90% train, 10% eval
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]
# Pass both train_dataset and eval_dataset to the Trainer
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,  # Add evaluation dataset here
    tokenizer = tokenizer,
    data_collator = data_collator,
)

<ipython-input-26-f9a85a1ca497>:7: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
model.push_to_hub("Samurai719214/mistral-mythology-storyteller")
tokenizer.push_to_hub("Samurai719214/mistral-mythology-storyteller")

adapter_model.safetensors:   0%|          | 0.00/27.3M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Samurai719214/mistral-mythology-storyteller/commit/271e181d1753e15ecbd13a1c9d56d69000065b92', commit_message='Upload tokenizer', commit_description='', oid='271e181d1753e15ecbd13a1c9d56d69000065b92', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Samurai719214/mistral-mythology-storyteller', endpoint='https://huggingface.co', repo_type='model', repo_id='Samurai719214/mistral-mythology-storyteller'), pr_revision=None, pr_num=None)